# 📘 Bloque 7: Redes Neuronales Multicapa (MLPs)

## 🔹 ¿Qué es un MLP?
Un **Perceptrón Multicapa (MLP, *Multi-Layer Perceptron*)** es una red neuronal **feedforward**, es decir, la información fluye siempre hacia adelante:

➡️ Entrada → Capas ocultas → Salida

Cada neurona realiza tres pasos básicos:

1. **Combinación lineal:**  
   \[
   z = \sum_{i=1}^n w_i x_i + b
   \]

2. **Función de activación:**  
   \[
   a = f(z)
   \]

3. **Propagación hacia la siguiente capa** con el valor de activación \(a\).

---

## 🔹 Componentes principales de un MLP

### 1. **Capas**
- **Entrada:** recibe los datos originales (ejemplo: píxeles de MNIST).  
- **Ocultas:** transforman los datos en representaciones más abstractas.  
- **Salida:** genera la predicción final (ejemplo: 10 neuronas para dígitos 0–9).  

---

### 2. **Funciones de activación**
- **ReLU:**  
  \[
  f(x) = \max(0, x)
  \]  
  ✅ Rápida, evita gradiente desapareciente, ideal para redes profundas.  

- **Sigmoid:**  
  \[
  f(x) = \frac{1}{1 + e^{-x}}
  \]  
  ✅ Útil para probabilidades. ❌ Problema: saturación en valores extremos.  

- **Tanh:**  
  \[
  f(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}
  \]  
  ✅ Salida entre -1 y 1, centrada en 0. ❌ Puede saturarse.  

---

### 3. **Inicialización de pesos**
- **Xavier (Glorot):** mantiene la varianza de señales estable → bueno para *sigmoid* y *tanh*.  
- **He (Kaiming):** ajustado para *ReLU*, compensa neuronas que se apagan.  

👉 Una buena inicialización evita gradientes explosivos o desaparecientes.  

---

### 4. **Optimizadores**
- **SGD:** simple descenso de gradiente estocástico. Lento pero estable.  
- **Adam:** adapta el learning rate con momentos del gradiente. Muy usado.  
- **RMSProp:** suaviza gradientes cuadrados, evita oscilaciones.  

---

### 5. **Regularización**
- **Dropout:** apaga neuronas aleatoriamente durante el entrenamiento → evita overfitting.  
- **Batch Normalization:** normaliza activaciones dentro de cada minibatch → entrenamiento más estable.  
- **Gradient Clipping:** limita valores de gradientes para evitar explosiones.  

---

### 6. **Learning Rate Scheduling**
El *learning rate* controla la velocidad de aprendizaje.  

- **StepLR:** reduce el learning rate cada cierto número de épocas.  
- **Cosine Annealing:** lo baja siguiendo una curva coseno.  
- **ReduceLROnPlateau:** lo baja cuando la pérdida deja de mejorar.  

---

## 🔹 Flujo completo de un MLP
1. **Forward pass:** datos → capas → salida.  
2. **Cálculo de pérdida:** compara predicción vs. verdad.  
3. **Backward pass:** cálculo de gradientes (*backpropagation*).  
4. **Optimización:** ajuste de pesos con el optimizador.  
5. **Entrenamiento por épocas:** la red mejora tras múltiples pasadas por los datos.  

---

📌 **En resumen:**  
Un **MLP aprende representaciones jerárquicas** de los datos mediante **transformaciones lineales + funciones de activación no lineales**.  
Cada componente (activación, inicialización, optimizador, regularización) influye en **qué tan bien y rápido aprende el modelo**.


**Importación y Configuración**

In [ ]:


# ================================================
# Bloque 7: Redes Neuronales Multicapa (MLPs)
# ================================================

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Usar CPU si está disponible
device = torch.device("cpu")
print("Usando:", device)

# Transformación de imágenes (28x28 -> tensor normalizado con media 0.5 y desv_standar 0.5)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])



# Cargar MNIST
train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

# Subconjunto pequeño para pruebas rápidas
# ⚠️ Puedes quitarlo si tienes buena máquina

# ================================================
# DataLoader en PyTorch
# ================================================

# train_dataset / test_dataset: datasets ya preparados (ej. MNIST)
# torch.utils.data.Subset: selecciona un subconjunto de datos
#   - range(10000) -> solo toma los primeros 10000 ejemplos del set de entrenamiento
#   - range(2000)  -> solo toma los primeros 2000 ejemplos del set de prueba

# DataLoader: crea un iterador que entrega los datos en **batches**
#   - batch_size=64 -> cada iteración devuelve 64 ejemplos
#   - shuffle=True  -> mezcla los datos cada época (solo para entrenamiento)
#   - shuffle=False -> no mezcla (útil para evaluación/testing)

# Ejemplo de dimensiones:
#   - train_loader iteración: x.shape = [64, 1, 28, 28], y.shape = [64]
#   - Cada batch se pasa al modelo para entrenamiento o evaluación


train_dataset = torch.utils.data.Subset(train_dataset, range(10000))
test_dataset = torch.utils.data.Subset(test_dataset, range(2000))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


Usando: cpu


**Definición del modelo**

In [ ]:
# Modelo básico MLP
class MLP(nn.Module):
    def __init__(self, input_size=784, hidden_size=128, output_size=10, activation="relu"):
        super(MLP, self).__init__()
        
        # Activaciones dinámicas
        if activation == "relu":
            act_fn = nn.ReLU()
        elif activation == "sigmoid":
            act_fn = nn.Sigmoid()
        elif activation == "tanh":
            act_fn = nn.Tanh()
        else:
            raise ValueError("Activación no soportada")
        
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            act_fn,
            nn.Linear(hidden_size, hidden_size),
            act_fn,
            nn.Linear(hidden_size, output_size)
        )
    
    def forward(self, x):
        x.view(x.size(0), -1)
        # ---------------------
        # Esta línea **aplana** cada imagen del batch en un vector 1D.
        # x.size(0) = tamaño del batch (número de ejemplos en la pasada actual)
        # -1 = PyTorch calcula automáticamente la otra dimensión necesaria
        # Ejemplo MNIST:
        #   x original: [64, 1, 28, 28]  -> 64 imágenes, 1 canal, 28x28 píxeles
        #   x después de view: [64, 784] -> cada imagen convertida en vector de 784 elementos
        # Esto es necesario antes de pasar los datos a capas lineales (nn.Linear)
        x = x.view(x.size(0), -1)  # Aplanar 28x28 → 784
        
        return self.net(x)

**Funciones de entrenamiento y evaluación**

In [3]:
def train(model, optimizer, criterion, loader):
    model.train()
    total_loss = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            pred = y_pred.argmax(dim=1)
            correct += (pred == y).sum().item()
            total += y.size(0)
    return correct / total


**Experimento con diferentes activaciones**

In [4]:
for act in ["relu", "sigmoid", "tanh"]:
    print(f"\n🔹 Entrenando con activación: {act.upper()}")
    model = MLP(activation=act).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(3):  # pocas épocas para pruebas
        loss = train(model, optimizer, criterion, train_loader)
        acc = evaluate(model, test_loader)
        print(f"Epoch {epoch+1} | Loss: {loss:.4f} | Test Acc: {acc:.4f}")



🔹 Entrenando con activación: RELU


Failed to parse CPUID


Epoch 1 | Loss: 0.7454 | Test Acc: 0.8370
Epoch 2 | Loss: 0.3439 | Test Acc: 0.8650
Epoch 3 | Loss: 0.2745 | Test Acc: 0.8895

🔹 Entrenando con activación: SIGMOID
Epoch 1 | Loss: 1.5557 | Test Acc: 0.7600
Epoch 2 | Loss: 0.5617 | Test Acc: 0.8555
Epoch 3 | Loss: 0.3577 | Test Acc: 0.8810

🔹 Entrenando con activación: TANH
Epoch 1 | Loss: 0.6569 | Test Acc: 0.8765
Epoch 2 | Loss: 0.2866 | Test Acc: 0.8905
Epoch 3 | Loss: 0.2180 | Test Acc: 0.9055


**Probar diferentes optimizadores**

In [5]:
optimizers = {
    "SGD": optim.SGD,
    "Adam": optim.Adam,
    "RMSProp": optim.RMSprop
}

for opt_name, opt_class in optimizers.items():
    print(f"\n🔹 Entrenando con optimizador: {opt_name}")
    model = MLP(activation="relu").to(device)
    optimizer = opt_class(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(3):
        loss = train(model, optimizer, criterion, train_loader)
        acc = evaluate(model, test_loader)
        print(f"Epoch {epoch+1} | Loss: {loss:.4f} | Test Acc: {acc:.4f}")



🔹 Entrenando con optimizador: SGD
Epoch 1 | Loss: 2.2893 | Test Acc: 0.2025
Epoch 2 | Loss: 2.2559 | Test Acc: 0.3160
Epoch 3 | Loss: 2.2244 | Test Acc: 0.3550

🔹 Entrenando con optimizador: Adam
Epoch 1 | Loss: 0.7327 | Test Acc: 0.8540
Epoch 2 | Loss: 0.3328 | Test Acc: 0.8815
Epoch 3 | Loss: 0.2663 | Test Acc: 0.8810

🔹 Entrenando con optimizador: RMSProp
Epoch 1 | Loss: 0.7400 | Test Acc: 0.8160
Epoch 2 | Loss: 0.3421 | Test Acc: 0.8220
Epoch 3 | Loss: 0.2729 | Test Acc: 0.8860


**Regularización con Dropout y BatchNorm**

In [6]:
class MLP_Reg(nn.Module):
    def __init__(self, input_size=784, hidden_size=128, output_size=10):
        super(MLP_Reg, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Dropout(0.5),  # 50% de neuronas apagadas
            nn.Linear(hidden_size, output_size)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.net(x)

print("\n🔹 Entrenando MLP con Dropout + BatchNorm")
model = MLP_Reg().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(3):
    loss = train(model, optimizer, criterion, train_loader)
    acc = evaluate(model, test_loader)
    print(f"Epoch {epoch+1} | Loss: {loss:.4f} | Test Acc: {acc:.4f}")



🔹 Entrenando MLP con Dropout + BatchNorm
Epoch 1 | Loss: 0.6956 | Test Acc: 0.8790
Epoch 2 | Loss: 0.3673 | Test Acc: 0.8825
Epoch 3 | Loss: 0.3043 | Test Acc: 0.8985


**Scheduler y Gradient Clipping**

In [7]:
print("\n🔹 Entrenando con Scheduler + Gradient Clipping")
model = MLP().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)

for epoch in range(5):
    model.train()
    total_loss = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)  # gradient clipping
        optimizer.step()
        total_loss += loss.item()
    scheduler.step()
    
    acc = evaluate(model, test_loader)
    print(f"Epoch {epoch+1} | Loss: {total_loss/len(train_loader):.4f} | Test Acc: {acc:.4f} | LR: {scheduler.get_last_lr()[0]:.5f}")



🔹 Entrenando con Scheduler + Gradient Clipping
Epoch 1 | Loss: 0.7311 | Test Acc: 0.8420 | LR: 0.01000
Epoch 2 | Loss: 0.3263 | Test Acc: 0.8900 | LR: 0.00500
Epoch 3 | Loss: 0.1960 | Test Acc: 0.8995 | LR: 0.00500
Epoch 4 | Loss: 0.1659 | Test Acc: 0.8710 | LR: 0.00250
Epoch 5 | Loss: 0.1250 | Test Acc: 0.9165 | LR: 0.00250
